#  Plot TFs DEG and activities in Visium spots


Note, run `general/G_clusterenrichment_TFs.r` on your DEGs before to estimate TF actitities.

In [1]:
library(dplyr)
library(reshape2)
library(plyr)
library(RColorBrewer)
library(ggplot2)
library(pheatmap)
library(cowplot)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************




## Load TF activities/enrichement file

In [2]:
TF_activities = read.csv('data/spatial/DEGs/TFs_activities.csv', stringsAsFactors = F)


TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities$id = paste(TF_activities$cl_name, TF_activities$Regulon)
head(TF_activities)

,cl_name,Regulon,NES,p.value,FDR,Size,id
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>
3,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,SP6_E,6.977937,2.995461e-12,2.878971e-11,139,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 SP6_E
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B
6,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,KDM5A_D,6.224759,4.822969e-10,3.667577e-09,185,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 KDM5A_D
8,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,IKZF2_E,5.859704,4.636940e-09,3.161342e-08,190,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 IKZF2_E
10,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,THAP1_C,5.828670,5.587086e-09,3.753654e-08,53,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 THAP1_C
11,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,ELF1_C,5.782228,7.371767e-09,4.905060e-08,56,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 ELF1_C


## Load TF agreement file to retrieve DE sign

In [3]:
TF_activities_agreement = read.csv('data/spatial/DEGs/TFactivities_and_expression_agreement.csv', stringsAsFactors = F, sep = '\t')
TF_activities_agreement$id = paste(TF_activities_agreement$cl, TF_activities_agreement$TF_activity_score)
TF_activities_agreement$annot = '+'
TF_activities_agreement$annot[ TF_activities_agreement$TF_expression == 'downregulated' ] = '-'
head(TF_activities_agreement)

,cl,TF,TF_expression,TF_expression_pvals_adj,TF_activity,TF_activity_pvals_adj,TF_activity_score,L1,id,annot
,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,KLF5,downregulated,1.30e-07,inactive_regulon,2.90e-04,KLF5_B,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 KLF5_B,-
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,EPAS1,upregulated,1.91e-02,inactive_regulon,5.04e-07,EPAS1_B,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 EPAS1_B,+
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SNAPC4,downregulated,4.41e-02,inactive_regulon,5.99e-03,SNAPC4_C,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SNAPC4_C,-
4,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,KLF6,upregulated,6.07e-20,active_regulon,2.63e-02,KLF6_B,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 KLF6_B,+
5,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,ID1,upregulated,7.36e-10,inactive_regulon,3.32e-03,ID1_E,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 ID1_E,+
6,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,POU5F1,upregulated,7.94e-08,inactive_regulon,6.31e-03,POU5F1_A,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 POU5F1_A,+


## Transfer DE sign

In [4]:
TF_activities$label = TF_activities_agreement$annot[ match(TF_activities$id, TF_activities_agreement$id) ]
TF_activities$label[ is.na(TF_activities$label) ] = ''
head(TF_activities)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
3,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,SP6_E,6.977937,2.995461e-12,2.878971e-11,139,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 SP6_E,
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B,
6,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,KDM5A_D,6.224759,4.822969e-10,3.667577e-09,185,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 KDM5A_D,
8,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,IKZF2_E,5.859704,4.636940e-09,3.161342e-08,190,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 IKZF2_E,
10,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,THAP1_C,5.828670,5.587086e-09,3.753654e-08,53,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 THAP1_C,
11,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,ELF1_C,5.782228,7.371767e-09,4.905060e-08,56,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 ELF1_C,


# Plot selected active TFs

In [5]:
# Rank TFs according to their activity
TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities = ddply(TF_activities, 'cl_name', function(df) data.frame(df, rank = 1:nrow(df)) )
head(TF_activities)
unique(TF_activities$cl_name)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B,,1
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,ZBTB7A_C,5.427291,5.721575e-08,5.209644e-07,90,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 ZBTB7A_C,,2
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MKX_E,4.783607,1.721773e-06,1.234680e-05,189,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MKX_E,,3
4,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MNT_C,4.624174,3.760938e-06,2.602569e-05,56,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MNT_C,,4
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,THAP11_C,4.229917,2.337772e-05,1.412872e-04,114,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 THAP11_C,,5
6,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,RFX5_AA,4.203369,2.629717e-05,1.575553e-04,11,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 RFX5_AA,,6


[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [6]:
# order clusters
TF_activities = rbind(subset(TF_activities, cl_name %in% 'cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423'),
                      subset(TF_activities, cl_name %in% 'cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423'),
                     subset(TF_activities, cl_name %in% 'cl_epi_basal_vs_epi_rest_limma_DEGs_20200423'))

In [7]:
head(TF_activities)
TFs = read.csv('data/scenic/huTF_census.txt', stringsAsFactors = F, header = F)$V1

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
208,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,RFX8_E,5.402977,6.554399e-08,9.754073e-07,173,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 RFX8_E,,1
209,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,ZSCAN1_E,4.955873,7.200627e-07,8.897918e-06,119,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 ZSCAN1_E,,2
210,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,HOXC11_D,4.620891,3.820947e-06,4.036787e-05,8,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 HOXC11_D,,3
211,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,HOXC4_E,4.146294,3.378998e-05,2.710421e-04,68,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 HOXC4_E,,4
212,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,GLI1_B,4.105610,4.032498e-05,3.171010e-04,55,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 GLI1_B,,5
213,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,FOXF1_E,3.789237,1.511110e-04,1.089258e-03,192,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 FOXF1_E,,6


In [8]:
TF_activities$Gene = sapply(strsplit(TF_activities$Regulon, '_'), head, 1)

In [9]:
# build matrix of TF activity scores to plot
TF_activities$value = TF_activities$NES
x = acast(TF_activities, Gene~cl_name, fill = 0)
colnames(x)

[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [10]:
x = x[, c(3, 2, 1) ]
colnames(x) = c('luminal',  'glandular', 'basal')


In [11]:
pheatmap(t(x[ c('GLI1', 'FOXJ1', 'HIF1A', 'CSRNP1', 'FOXO3', 'HEY1', 'ELF1') , ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F,
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/selected_TFs_activities.pdf')

# Plot top 10 active TFs

In [12]:
# Rank TFs according to their activity
TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities = ddply(TF_activities, 'cl_name', function(df) data.frame(df, rank = 1:nrow(df)) )
head(TF_activities)
unique(TF_activities$cl_name)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,rank,Gene,value,rank.1
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<int>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B,,1,SMAD7,6.608119,1
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,ZBTB7A_C,5.427291,5.721575e-08,5.209644e-07,90,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 ZBTB7A_C,,2,ZBTB7A,5.427291,2
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MKX_E,4.783607,1.721773e-06,1.234680e-05,189,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MKX_E,,3,MKX,4.783607,3
4,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MNT_C,4.624174,3.760938e-06,2.602569e-05,56,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MNT_C,,4,MNT,4.624174,4
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,THAP11_C,4.229917,2.337772e-05,1.412872e-04,114,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 THAP11_C,,5,THAP11,4.229917,5
6,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,RFX5_AA,4.203369,2.629717e-05,1.575553e-04,11,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 RFX5_AA,,6,RFX5,4.203369,6


[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [13]:
# order clusters
TF_activities = rbind(subset(TF_activities, cl_name %in% 'cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423'),
                      subset(TF_activities, cl_name %in% 'cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423'),
                     subset(TF_activities, cl_name %in% 'cl_epi_basal_vs_epi_rest_limma_DEGs_20200423'))

In [14]:
head(TF_activities)
TFs = read.csv('data/scenic/huTF_census.txt', stringsAsFactors = F, header = F)$V1

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,rank,Gene,value,rank.1
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<int>
208,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,RFX8_E,5.402977,6.554399e-08,9.754073e-07,173,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 RFX8_E,,1,RFX8,5.402977,1
209,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,ZSCAN1_E,4.955873,7.200627e-07,8.897918e-06,119,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 ZSCAN1_E,,2,ZSCAN1,4.955873,2
210,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,HOXC11_D,4.620891,3.820947e-06,4.036787e-05,8,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 HOXC11_D,,3,HOXC11,4.620891,3
211,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,HOXC4_E,4.146294,3.378998e-05,2.710421e-04,68,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 HOXC4_E,,4,HOXC4,4.146294,4
212,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,GLI1_B,4.105610,4.032498e-05,3.171010e-04,55,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 GLI1_B,,5,GLI1,4.105610,5
213,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423,FOXF1_E,3.789237,1.511110e-04,1.089258e-03,192,cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423 FOXF1_E,,6,FOXF1,3.789237,6


In [15]:
# Select top 10 ranked genes
TF_activities$Gene = sapply(strsplit(TF_activities$Regulon, '_'), head, 1)
top30 = subset(TF_activities, rank %in% 1:30 & NES > 0)$Gene

In [16]:
# build matrix of TF activity scores to plot
TF_activities$value = TF_activities$NES
x = acast(subset(TF_activities, Gene %in% top30), Gene~cl_name, fill = 0)
colnames(x)

[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [17]:
x = x[, c(3, 2, 1) ]
colnames(x) = c('luminal',  'glandular', 'basal')
pheatmap(t(x[ top30, ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F, 
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/top30_TFs_activities.pdf')


In [18]:
pheatmap(t(x[ c('GLI1', 'FOXJ1', 'HIF1A', 'CSRNP1', 'HEY1', 'ELF1') , ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F,
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/selected_TFs_activities.pdf')

# Plot top 10 active TFs - matching expression

In [19]:
default_TF_activities = TF_activities

In [20]:
TF_activities = subset(TF_activities, label != '')

In [21]:
# Rank TFs according to their activity
TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities$rank = NULL
TF_activities = ddply(TF_activities, 'cl_name', function(df) data.frame(df, rank = 1:nrow(df)) )
head(TF_activities)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,Gene,value,rank.1,rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SNAPC4_C,-3.178725,1.479241e-03,5.986169e-03,30,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SNAPC4_C,-,SNAPC4,-3.178725,63,1
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,KLF5_B,-4.048474,5.155264e-05,2.900360e-04,12,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 KLF5_B,-,KLF5,-4.048474,73,2
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,EPAS1_B,-5.434550,5.493486e-08,5.035089e-07,43,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 EPAS1_B,+,EPAS1,-5.434550,80,3
4,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,HIF1A_AA,4.971304,6.650424e-07,3.638018e-06,59,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 HIF1A_AA,+,HIF1A,4.971304,9,1
5,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,NFE2L1_B,3.710300,2.070142e-04,7.327560e-04,25,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 NFE2L1_B,+,NFE2L1,3.710300,33,2
6,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423,FOXO3_AA,3.584532,3.376827e-04,1.151115e-03,19,cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423 FOXO3_AA,+,FOXO3,3.584532,36,3


In [22]:
# order clusters
TF_activities = rbind(subset(TF_activities, cl_name %in% 'cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423'),
                      subset(TF_activities, cl_name %in% 'cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423'),
                     subset(TF_activities, cl_name %in% 'cl_epi_basal_vs_epi_rest_limma_DEGs_20200423'))

In [23]:
# Select top 10 ranked genes
TF_activities$Gene = sapply(strsplit(TF_activities$Regulon, '_'), head, 1)
top30 = subset(TF_activities, rank %in% 1:30 & NES > 0)$Gene

In [24]:
# build matrix of TF activity scores to plot
TF_activities$value = TF_activities$NES
x = acast(subset(TF_activities, Gene %in% top30), Gene~cl_name, fill = 0)
colnames(x) 

[1] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[2] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [25]:
x = x[, c(2, 1) ]
colnames(x) = c('luminal',  'glandular')
pheatmap(t(x[ top30, ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F, 
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/top30_TFs_activities_filtered_TFisDE.pdf')


# Plot top 10 active TFs - curated TFs - confidence AA, A, B

In [26]:
TF_activities = default_TF_activities[ grep('_E$', default_TF_activities$Regulon, invert = T) , ]
TF_activities = TF_activities[ grep('_D$', TF_activities$Regulon, invert = T) , ]
TF_activities = TF_activities[ grep('_C$', TF_activities$Regulon, invert = T) , ]

In [27]:
# Rank TFs according to their activity
TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities$rank = NULL
TF_activities = ddply(TF_activities, 'cl_name', function(df) data.frame(df, rank = 1:nrow(df)) )
head(TF_activities)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,Gene,value,rank.1,rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B,,SMAD7,6.608119,1,1
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,RFX5_AA,4.203369,2.629717e-05,1.575553e-04,11,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 RFX5_AA,,RFX5,4.203369,6,2
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,IRF4_B,3.390199,6.984185e-04,3.088215e-03,8,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 IRF4_B,,IRF4,3.390199,10,3
4,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SRF_A,3.026489,2.474115e-03,9.215843e-03,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SRF_A,,SRF,3.026489,15,4
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,WT1_A,3.021510,2.515176e-03,9.332448e-03,26,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 WT1_A,,WT1,3.021510,16,5
6,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,ETS1_AA,2.544535,1.094234e-02,3.343090e-02,63,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 ETS1_AA,,ETS1,2.544535,23,6


In [28]:
# order clusters
TF_activities = rbind(subset(TF_activities, cl_name %in% 'cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423'),
                      subset(TF_activities, cl_name %in% 'cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423'),
                     subset(TF_activities, cl_name %in% 'cl_epi_basal_vs_epi_rest_limma_DEGs_20200423'))

In [29]:
# Select top 10 ranked genes
TF_activities$Gene = sapply(strsplit(TF_activities$Regulon, '_'), head, 1)
top30 = subset(TF_activities, rank %in% 1:30 & NES > 0)$Gene

In [30]:
# build matrix of TF activity scores to plot
TF_activities$value = TF_activities$NES
x = acast(subset(TF_activities, Gene %in% top30), Gene~cl_name, fill = 0)
colnames(x)

[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [31]:
x = x[, c(3, 2, 1) ]
colnames(x) = c('luminal',  'glandular', 'basal')
pheatmap(t(x[ top30, ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F, 
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/top30_TFs_activities_filtered_curatedTFs.pdf')


# Plot top 10 active TFs - DE or curated

In [32]:
idx = unique(c( which(default_TF_activities$label != ''),
               grep('_E$', default_TF_activities$Regulon, invert = T),
               grep('_D$', default_TF_activities$Regulon, invert = T),
               grep('_C$', default_TF_activities$Regulon, invert = T)
              )) %>% sort(.)
TF_activities = default_TF_activities[ idx, ]

In [33]:
# Rank TFs according to their activity
TF_activities = TF_activities[order(TF_activities$NES, decreasing = T), ]
TF_activities$rank = NULL
TF_activities = ddply(TF_activities, 'cl_name', function(df) data.frame(df, rank = 1:nrow(df)) )
head(TF_activities)

,cl_name,Regulon,NES,p.value,FDR,Size,id,label,Gene,value,rank.1,rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>
1,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,SMAD7_B,6.608119,3.892325e-11,5.281350e-10,14,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 SMAD7_B,,SMAD7,6.608119,1,1
2,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,ZBTB7A_C,5.427291,5.721575e-08,5.209644e-07,90,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 ZBTB7A_C,,ZBTB7A,5.427291,2,2
3,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MKX_E,4.783607,1.721773e-06,1.234680e-05,189,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MKX_E,,MKX,4.783607,3,3
4,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,MNT_C,4.624174,3.760938e-06,2.602569e-05,56,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 MNT_C,,MNT,4.624174,4,4
5,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,THAP11_C,4.229917,2.337772e-05,1.412872e-04,114,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 THAP11_C,,THAP11,4.229917,5,5
6,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423,RFX5_AA,4.203369,2.629717e-05,1.575553e-04,11,cl_epi_basal_vs_epi_rest_limma_DEGs_20200423 RFX5_AA,,RFX5,4.203369,6,6


In [34]:
# order clusters
TF_activities = rbind(subset(TF_activities, cl_name %in% 'cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423'),
                      subset(TF_activities, cl_name %in% 'cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423'),
                     subset(TF_activities, cl_name %in% 'cl_epi_basal_vs_epi_rest_limma_DEGs_20200423'))

In [35]:
# Select top 10 ranked genes
TF_activities$Gene = sapply(strsplit(TF_activities$Regulon, '_'), head, 1)
top30 = subset(TF_activities, rank %in% 1:30 & NES > 0)$Gene

In [36]:
# build matrix of TF activity scores to plot
TF_activities$value = TF_activities$NES
x = acast(subset(TF_activities, Gene %in% top30), Gene~cl_name, fill = 0)
colnames(x) 

[1] "cl_epi_basal_vs_epi_rest_limma_DEGs_20200423"    
[2] "cl_epi_glandular_vs_epi_rest_limma_DEGs_20200423"
[3] "cl_epi_luminal_vs_epi_rest_limma_DEGs_20200423"

In [37]:
x = x[, c(3, 2, 1) ]
colnames(x) = c('luminal',  'glandular', 'basal')
pheatmap(t(x[ top30, ]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F, 
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/top30_TFs_activities_filtered_curatedTFs_TFisDE.pdf')


# Plot top 10 DE TFs

In [38]:
TFs = read.csv('data/scenic/huTF_census.txt', stringsAsFactors = F, header = F)$V1

DE_gland = read.csv('data/spatial/DEGs/epi_glandular_vs_epi_rest_limma_DEGs_20200423.csv', stringsAsFactors = F)
DE_luminal = read.csv('data/spatial/DEGs/epi_luminal_vs_epi_rest_limma_DEGs_20200423.csv', stringsAsFactors = F)
DE_basal = read.csv('data/spatial/DEGs/epi_basal_vs_epi_rest_limma_DEGs_20200423.csv', stringsAsFactors = F)

DE = list(luminal=DE_luminal,
         gland=DE_gland,
         basal=DE_basal)

DE = lapply(DE, function(x){
    x = subset(x, Gene %in% TFs)
    x = x[ order(x$logFC, decreasing = T) ,]
    x$rank = 1:nrow(x)
    return(x)
})

In [39]:
DE_df = melt(DE, id.vars = names(DE[[1]]) )
head(DE_df)

,Gene,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest,rank,L1
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
1,HAND2,0.2604295,2.649793e-05,0.001369203,1.352028,1.091599,0.972603,0.924471,1,luminal
2,NR2F2,0.2403253,1.647070e-04,0.006357223,1.098132,0.857806,0.904110,0.867069,2,luminal
3,ESR1,0.2208052,1.307507e-04,0.005293044,0.797663,0.576857,0.780822,0.749245,3,luminal
4,CUX1,0.2117044,5.227082e-04,0.016023274,0.964216,0.752511,0.890411,0.842900,4,luminal
5,OSR2,0.2029021,1.112307e-03,0.028315565,1.640227,1.437325,0.958904,0.966767,5,luminal
6,HES4,0.1834099,1.105987e-03,0.028230060,0.793668,0.610258,0.849315,0.761329,6,luminal


In [40]:
# Select top 30 ranked genes
top30 = subset(DE_df, rank %in% 1:30)$Gene #%>% unique(.)
top30

[1] "HAND2"   "NR2F2"   "ESR1"    "CUX1"    "OSR2"    "HES4"    "TEAD2"  
 [8] "PAX8"    "GTF3A"   "HOXA10"  "PRRX1"   "HOXA11"  "TCF12"   "SOX4"   
[15] "HOXA9"   "TWIST1"  "SNAPC4"  "SNAI2"   "PGR"     "TEAD3"   "FOXJ1"  
[22] "TSHZ3"   "MXD4"    "MEIS1"   "KLF16"   "PRDM1"   "AR"      "ZNF511" 
[29] "ZNF451"  "RFXANK"  "KLF6"    "ELF3"    "ID1"     "JUND"    "KLF5"   
[36] "ELK4"    "RORC"    "JUNB"    "POU5F1"  "CASZ1"   "HIF1A"   "ID4"    
[43] "FOSL2"   "HOXB6"   "BCL6"    "ATF4"    "PAX8"    "ZNF600"  "CEBPD"  
[50] "XBP1"    "DLX5"    "ZNF787"  "HOXB3"   "HOXB2"   "SMAD9"   "MECOM"  
[57] "LRRFIP2" "ZNF217"  "MAFF"    "KLF13"   "SOX9"    "ATOH8"   "NFKBIA" 
[64] "EPAS1"   "YBX3"    "ETS2"    "TSC22D1" "XBP1"    "TCF7L1"  "ZNF280D"
[71] "CEBPA"   "ZNF22"   "NFIA"    "ZNF417"  "ZNF362"  "JUN"     "NFKBIZ" 
[78] "KLF15"   "MAFG"    "ZNF680"  "MSX1"    "AHR"     "STAT3"   "ZNF445" 
[85] "CEBPD"   "PEG3"    "KLF10"   "ZNF253"  "CSRNP1"  "RXRB"

In [41]:
# build matrix of TF activity scores to plot
DE_df$value = DE_df$logFC
x = acast(subset(DE_df, Gene %in% top30), Gene~L1, fill = 0)
pheatmap(t(x[top30, names(DE)]), cellheight = 10, cellwidth = 10, cluster_rows = F, cluster_cols = F, 
         color = colorRampPalette(c(brewer.pal(n = 5, name = 'Blues')[4], "white", brewer.pal(n = 5, name = 'Reds')[4]))(50),
        filename = 'data/spatial/DEGs/top30_TFs_DE.pdf')
